<a href="https://colab.research.google.com/github/Katlego-TheDataScientist/Attendance_Register/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install flask flask-ngrok python-dotenv supabase apscheduler pyngrok
!pip install flask_cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.15
    Uninstalling aiohttp-3.11.15:
      Successfully uninstalled aiohttp-3.11.15


In [ ]:
from flask import Flask, jsonify, request, render_template
from flask_cors import CORS
from datetime import datetime
from apscheduler.schedulers.background import BackgroundScheduler
from supabase import create_client, Client
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

app = Flask(__name__, static_folder='static', template_folder='templates')
CORS(app)

# ---------------------------
# Email Sending (smtplib)
# ---------------------------
def send_email(to_email, subject, body_html):
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = os.getenv("SMTP_EMAIL")
    msg["To"] = to_email

    part = MIMEText(body_html, "html")
    msg.attach(part)

    try:
        server = smtplib.SMTP(os.getenv("SMTP_SERVER"), int(os.getenv("SMTP_PORT")))
        server.starttls()
        server.login(os.getenv("SMTP_EMAIL"), os.getenv("SMTP_PASSWORD"))
        server.sendmail(msg["From"], msg["To"], msg.as_string())
        server.quit()
        print(f"Email sent to {to_email}")
    except Exception as e:
        print(f"Failed to send email to {to_email}: {e}")


# ---------------------------
# Monthly Attendance Reports
# ---------------------------
def send_monthly_reports():
    students = supabase.table("students").select("*").execute().data
    for student in students:
        sid = student["id"]
        email = student.get("parent_email")
        if not email:
            continue

        records = supabase.table("attendances").select("*").eq("student_id", sid).execute().data
        total = len(records)
        present = sum(1 for r in records if r["attended"])
        percent = round((present / total) * 100, 2) if total else 0

        html = f"""
        <h3>Monthly Attendance Report for {student['first_name']} {student['last_name']}</h3>
        <p><strong>Total Sessions:</strong> {total}</p>
        <p><strong>Present:</strong> {present}</p>
        <p><strong>Attendance:</strong> {percent}%</p>
        """

        send_email(email, "Monthly Attendance Report", html)


# ---------------------------
# API Routes
# ---------------------------

@app.route("/")
def home():
    return jsonify({"message": "Attendance system is running"})


@app.route("/students", methods=["POST"])
def register_student():
    data = request.get_json()
    response = supabase.table("students").insert(data).execute()
    return jsonify(response.data), 201


@app.route("/sessions", methods=["POST"])
def create_session():
    data = request.get_json()
    data["date"] = datetime.utcnow().isoformat()
    response = supabase.table("sessions").insert(data).execute()
    return jsonify(response.data), 201


@app.route("/attendance", methods=["POST"])
def mark_attendance():
    data = request.get_json()
    response = supabase.table("attendances").insert(data).execute()
    return jsonify(response.data), 201


@app.route("/attendance/<int:student_id>", methods=["GET"])
def get_attendance(student_id):
    response = supabase.table("attendances").select("*").eq("student_id", student_id).execute()
    return jsonify(response.data), 200


@app.route("/report/<int:student_id>", methods=["GET"])
def attendance_report(student_id):
    records = supabase.table("attendances").select("*").eq("student_id", student_id).execute().data
    total = len(records)
    present = sum(1 for r in records if r["attended"])
    percent = round((present / total) * 100, 2) if total else 0

    report = {
        "total_sessions": total,
        "present": present,
        "attendance_percent": percent,
        "records": records
    }

    return jsonify(report), 200

@app.route("/")
def index():
    return render_template("index.html")

# ---------------------------
# Scheduler
# ---------------------------
scheduler = BackgroundScheduler()
scheduler.add_job(send_monthly_reports, 'cron', day=28, hour=17, minute=59)  # Adjust date as needed
scheduler.start()

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=int(os.getenv("PORT", 5000)))


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
